<a href="https://colab.research.google.com/github/nithin-srivatsa/PortfolioOptimization/blob/main/Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1

# Store average return (parameter r_i in portfolio optimization model)
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

# Model 1

In [ ]:
model = Model("PortfolioOptimization")

# Variable
w = {i: model.addVar(lb=0.0, ub=1.0, name=f"weight_{i}") for i in tickind}

In [ ]:
# Constraint: The sum of weights must equal 1
model.addConstr(sum(w[i] for i in tickind) == 1, name="Weights_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Constraint: Each individual weight must be greater than or equal to 0
for i in tickind:
    model.addConstr(w[i] >= 0, name=f"Non_negative_constraint_{i}")

In [ ]:
# Constraint: The sum of weighted returns must be greater than 0.005
model.addConstr(sum(w[i] * avg_return[i] for i in tickind) >= 0.005, name="Sum_of_Weighted_Returns_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Objective function: Minimize the portfolio variance
portfolio_variance = sum(w[i] * w[j] * C[i, j] for i in tickind for j in tickind)
model.setObjective(portfolio_variance, GRB.MINIMIZE)

In [ ]:
# Solve the model
model.update()
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x3bac9f13
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.93770406e+03 -2.93770406e+03  4.0

In [ ]:
# Access the solution
if model.status == GRB.OPTIMAL:
    for i in tickind:
        print(f"{tickers[i]}: {round(w[i].x * 100, 3)}%")
    print(f"Total Portfolio Risk: {model.objVal}")

MSFT: 23.712%
GS: 2.586%
PG: 0.0%
SCHP: 73.702%
Total Portfolio Risk: 0.00017749363882836112


# Model 2

In [ ]:
model_allstocks = Model("PortfolioOptimizationAcrossAllStocks")

# Collect indexes of all tickers in portfolio
tickind2 =[];
for t in tickers:
    tickind2.append(tickers.index(t))

w2 = {i: model_allstocks.addVar(lb=0.0, ub=1.0, name=f"weight_{i}") for i in tickind2}

In [ ]:
# Constraint: The sum of weights must equal 1
model_allstocks.addConstr(sum(w2[i] for i in tickind2) == 1, name="Weights_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Constraint: Each individual weight must be greater than or equal to 0
for i in tickind2:
    model_allstocks.addConstr(w2[i] >= 0, name=f"Non_negative_constraint_{i}")

In [ ]:
# Constraint: The sum of weighted returns must equal 1
model_allstocks.addConstr(sum(w2[i] * avg_return[i] for i in tickind2) >= 0.005, name="Sum_of_Weighted_Returns_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Objective function: Minimize the portfolio variance
portfolio_variance_allstocks = sum(w2[i] * w2[j] * C[i, j] for i in tickind2 for j in tickind2)
model_allstocks.setObjective(portfolio_variance_allstocks, GRB.MINIMIZE)

In [ ]:
# Solve the model
model_allstocks.update()
model_allstocks.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0x275c503d
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.10520633e-17 -

In [ ]:
tolerance = 1e-6  # Adjust the tolerance level as needed

# Access the solution
if model_allstocks.status == GRB.OPTIMAL:
    for i in tickind2:
        if w2[i].x > tolerance:
            print(f"{tickers[i]}: {round(w2[i].x * 100, 3)}%")
    print(f"Total Portfolio Risk: {model_allstocks.objVal}")

ABBV: 1.135%
ABMD: 0.667%
ATVI: 1.369%
ANET: 1.175%
AIZ: 2.223%
ATO: 2.917%
BBY: 0.26%
CME: 1.275%
ED: 0.018%
DRI: 0.669%
RE: 0.235%
GWW: 0.777%
HAS: 0.246%
HCA: 2.955%
HUM: 2.267%
INFO: 5.173%
ICE: 1.418%
KEYS: 2.217%
LHX: 0.0%
LLY: 0.852%
LMT: 0.066%
PSX: 0.799%
PNC: 3.229%
BND: 68.056%
Total Portfolio Risk: 2.878503835027502e-05


# Model 3

In [ ]:
#MODEL 3:

dict3 = {}
for i,n in enumerate(tickers):
    dict3[i] = i


# Define model and parameters.
mod3 = Model()

# Define decision variables
# w_i is the weight on asset i
w = mod3.addVars(len(dict3))
x = mod3.addVars(len(dict3), vtype = GRB.BINARY)

# Writing expected return should atleast be 0.5
mod3.addConstr(sum(w[i]*avg_return[dict3[i]] for i in range(len(dict3))) >= 0.005)

# Total weight constraint should be equal to 1
mod3.addConstr(sum(w[i] for i in range(len(dict3))) == 1)

# Non negativity
for i in range(len(dict3)):
    mod3.addConstr(w[i] >= 0)

# x and w relation
for i in range(len(dict3)):
    mod3.addConstr(w[i] <= x[i])

# Total sum should be less than 4
mod3.addConstr(sum(x[i] for i in range(len(dict3))) <= 4)


# Create the objective function, and set it to be maximized.
mod3.setObjective(sum(sum(w[i]*w[j]*C[dict3[i],dict3[j]] for i in range(len(dict3))) for j in range(len(dict3))), GRB.MINIMIZE)

mod3.update()

mod3.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x3907714c
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

In [ ]:
# Check if the optimization was successful
if mod3.status == GRB.OPTIMAL:
    # Display the asset allocation
    print("Asset Allocations:")
    for i in range(len(dict3)):
        if x[i].x > 0.5:  # if the asset is included in the portfolio
            ticker = tickers[i]
            weight = w[i].x
            print(f"Asset: {ticker}, Weight: {weight:.2%}")

    # Display the total expected return
    total_return = sum(w[i].x * avg_return[dict3[i]] for i in range(len(dict3)))
    print(f"\nTotal Expected Return: {total_return:.2%}")
    total_variance = sum(sum(w[i].x * w[j].x * C[dict3[i], dict3[j]] for i in range(len(dict3))) for j in range(len(dict3)))

    # Display the total variance
    print(f"Total Variance of the Portfolio: {total_variance}")

else:
    print("No optimal solution was found.")

Asset Allocations:
Asset: CME, Weight: 12.64%
Asset: LLY, Weight: 7.55%
Asset: NVDA, Weight: 4.38%
Asset: BND, Weight: 75.44%

Total Expected Return: 0.50%
Total Variance of the Portfolio: 6.753470760728118e-05


## Inference 1

**A. For Model 1, this is the optimal risk (i.e. the optimal objective function value), solver time, and the weight on each of the four stocks.**

In [ ]:
print("Solver time for model 1 is %f" % model.Runtime, "seconds")
print("The weight of each of the four stocks is -")
if model.status == GRB.OPTIMAL:
    for i in tickind:
        print(f"{tickers[i]}: {round(w[i].x * 100, 3)}%")
    print(f"Total Portfolio Variance: {round(model.objVal*100,4)}%")

Solver time for model 1 is 0.014339 seconds
The weight of each of the four stocks is -
MSFT: 23.712%
GS: 2.586%
PG: 0.0%
SCHP: 73.702%
Total Portfolio Variance: 0.0177%


**B. For Model 2, this is the optimal risk and solver time.**

In [ ]:
print("Solver time for model 2 is %f" % model_allstocks.Runtime, "seconds")
print(f"Total Portfolio Variance: {round(model_allstocks.objVal*100,4)}%")

Solver time for model 2 is 0.051196 seconds
Total Portfolio Variance: 0.0029%


**C. For Model 3, this is the optimal risk, solver time, and the ticker and weight on each of the
four stocks selected by the model**

In [ ]:
# Check if the optimization was successful
if mod3.status == GRB.OPTIMAL:
    # Iterate over the indices of tickers
    for i in tickind2:
        # Check if the binary variable x[i] is set to 1
        if x[i].x == 1:
            # Print the selected tickers and their weights
            print(f"{tickers[i]}: {round(w[i].x * 100, 3)}%")

    # Print the total portfolio variance
    print(f"Total Portfolio Variance: {round(mod3.objVal*100, 4)}%")

CME: 12.641%
LLY: 7.548%
NVDA: 4.375%
BND: 75.436%
Total Portfolio Variance: 0.0068%


### Model 1 vs Model 2
* **Model 2 has lower optimal risk** - Model 2 has a risk of 0.0029% as compared to Model 1, which has the risk of 0.0177%
* **Model 2 is more diversified** - With the increase in number of avaiable stocks from 4 to 390, the porfolio became more diversified. This diversification helps reduce risk as we can now choose stocks (more than 4) which are relatively less correlated and whose combination provides us with a sufficient return. This reduces the volatility in stocks, therefore having lower variance/risk

### Model 2 vs Model 3
* **Model 2 has lower optimal risk** - Model 2 has a risk of 0.0029% as comparedto Model 1, which has the risk of 0.0068%
* **Model 3 is less diversified** - Model 3 can hold a portfolio of 1,2,3 or 4 stocks at best as compare to Model 2 which has 390 stocks at its disposal to create an optimal portfolio. The freedom to create a porfolio of more diverse assets (with a combination of more stocks that have lower correlation) is likely the reason for Model 2 outperforming Model 3 in risk.

## Model 3 with Time Limit

In [ ]:

#MODEL 3 OUTPUT:

print("Optimal risk for model is %f" % mod3.ObjVal)
print("Solver time for model is %f" % mod3.Runtime)

Optimal risk for model is 0.000068
Solver time for model is 45.107383


In [ ]:
#REDEFINING THE MODEL 3 WITH TIME LIMIT

# Define model and parameters.
mod4 = Model()
mod4.Params.TimeLimit = 3.00

# Define decision variables
# w_i is the weight on asset i
w = mod4.addVars(len(dict3))
x = mod4.addVars(len(dict3), vtype = GRB.BINARY)

# Writing expected return should atleast be 0.5
mod4.addConstr(sum(w[i]*avg_return[dict3[i]] for i in range(len(dict3))) >= 0.005)

# Total weight constraint should be equal to 1
mod4.addConstr(sum(w[i] for i in range(len(dict3))) == 1)

# Non negativity
for i in range(len(dict3)):
    mod4.addConstr(w[i] >= 0)

# x and w relation
for i in range(len(dict3)):
    mod4.addConstr(w[i] <= x[i])

# Total sum should be less than 4
mod4.addConstr(sum(x[i] for i in range(len(dict3))) <= 4)


# Create the objective function, and set it to be maximized.
mod4.setObjective(sum(sum(w[i]*w[j]*C[dict3[i],dict3[j]] for i in range(len(dict3))) for j in range(len(dict3))), GRB.MINIMIZE)

mod4.update()

mod4.optimize()

Set parameter TimeLimit to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x3907714c
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Obje

In [ ]:
print("Optimal risk  for model is %f" % mod4.ObjVal)
print("Solver time for model is %f" % mod4.Runtime)

Optimal risk  for model is 0.000068
Solver time for model is 3.014579


Solver time for the updated model 3 is 3 seconds, which is much less than the previous model 3 that took 45 seconds but the optimal risk is still the same value when looked at as a whole number. Perhaps it implies that for this solution, there is diminishing , negligible returns beyond solving at a certain point, because there is actually a tiny difference between the two values. Overall, the result is slightly worse for the 3 second model since it was cut off.

Optimal model: **Best objective 6.75347e-05**

3 Second model: **Best objective 6.758574167459e-05**

If we observe Gurobi Output, we see that the solution for Optimal Model was found on Solution 7, whereas for the 3 second Model, it was found on Solution 6. So it took considerably longer for the final solution, which changed the result very very marginally. This could be seen as the utility of setting the right cut-off point for a model.

In [ ]:
## #3B UPDATED UPGRADED MODEL 3

# Define model and parameters.
mod5 = Model()
mod5.Params.MIPGap = 0.1

# Define decision variables
# w_i is the weight on asset i
w = mod5.addVars(len(dict3))
x = mod5.addVars(len(dict3), vtype = GRB.BINARY)

# Writing expected return should atleast be 0.5
mod5.addConstr(sum(w[i]*avg_return[dict3[i]] for i in range(len(dict3))) >= 0.005)

# Total weight constraint should be equal to 1
mod5.addConstr(sum(w[i] for i in range(len(dict3))) == 1)

# Non negativity
for i in range(len(dict3)):
    mod5.addConstr(w[i] >= 0)

# x and w relation
for i in range(len(dict3)):
    mod5.addConstr(w[i] <= x[i])

# Total sum should be less than 4
mod5.addConstr(sum(x[i] for i in range(len(dict3))) <= 4)


# Create the objective function, and set it to be maximized.
mod5.setObjective(sum(sum(w[i]*w[j]*C[dict3[i],dict3[j]] for i in range(len(dict3))) for j in range(len(dict3))), GRB.MINIMIZE)

mod5.update()

mod5.optimize()

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x3907714c
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.03s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objec

In [ ]:
print("Optimal risk for model is %f" % mod5.ObjVal)
print("Solver time for model is %f" % mod5.Runtime)

Optimal risk for model is 0.000068
Solver time for model is 25.075235


Solver time for the updated model 3 is 25.33, but still manages to get close to a very similar optimal risk.

This might indicate that for this problem, there is no major difference between an optimality gap of 10%, versus 0%.

However, the overall in this case is still slightly higher than the optimal risk when the program is allowed to run without restrictions. The amount of change though, is negligible. The best bound, however is slightly better for the Optimal Model.


10% Gap Model: 6.753470760728e-05, best bound 6.114269814352e-05


Optimal model: 6.753470760728e-05, best bound 6.753470760728e-05

Overall, this exercise higlights the advantages of using an optimality gap or a time cut off. It may be beneficial in certain cases to get close, but not waste further computational resources in finding "THE" optimal solution.
